In [1]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
from ipynb.fs.full.reading_files import *
from ipynb.fs.full.rate_prop_calculation import *
from ipynb.fs.full.cost_benefit import *
from ipynb.fs.full.choose_jurisdiction import *

C:\Users\dsharma1\OneDrive - University of South Florida\My files\Paper health\Hanisha\Data\CTDE\choose_jurisdiction.ipynb:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  },


In [3]:
age_begin = 13
age_end = 100
risk_groups = ['HM', 'HF', 'MSM']
num_risk = 3

all_jur = 10
age_groups = 88
number_of_risk_groups = 3
number_of_compartments = 22
dt = 1/12

num_age = 88
num_comp = number_of_compartments-2

prep_efficiency = 0.99

unaware_index = (1,5,9,13,17)
aware_no_care_index = (2,6,10,14,18)
ART_VLS_index = (3,4,7,8,11,12,15,16,19,20)
VLS_index = (4,8,12,16,20)


pop_growth_rate = 0

gamma = np.array([[0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1]])

scaling_factor_dropout = np.array([[1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0]])

all_jurisdictions = [6001,6037,6059,6065,6067,6071,6073,6]

num_jur = len(all_jurisdictions)

cluster1 = [6001]
cluster2 = [6037]
cluster3 = [6059]
cluster4 = [6065]
cluster5 = [6067]
cluster6 = [6071]
cluster7 = [6073]
cluster8 = [6]

cluster1_index = np.array([0])
cluster2_index = np.array([1])
cluster3_index = np.array([2])
cluster4_index = np.array([3])
cluster5_index = np.array([4])
cluster6_index = np.array([5])
cluster7_index = np.array([6])
cluster8_index = np.array([7])

In [5]:
ltc_excel ='./input_files/CareContinuum-by_jur 7-26-2021 HT V2.xlsx'

ltc_risk = ltc_prep_values(ltc_excel, all_jurisdictions)  # linked to care rate

In [6]:
new_infections_data = read_new_inf_input(age_begin, age_end, risk_groups) # new infection data from excel

M_x1_y1_i = M_x1_y1_value(new_infections_data)

In [7]:
death_prob_data = read_death_rates(age_begin, age_end, risk_groups)

death_rate_risk_age_compartments = np.zeros((num_jur, number_of_risk_groups, age_groups, number_of_compartments))

for risk in range(len(risk_groups)):
    for age in range(age_groups):
        death_rate_risk_age_compartments[:,risk,age,] = calculate_deaths_vector(number_of_compartments, risk_groups, risk, age, death_prob_data).reshape(22)

In [8]:
def get_action(min_, max_, val):
    new = min_ + (max_ - min_) * (val + 1)/ 2 
    return new

In [9]:
def change_action_range(action):
    for i in range(3):
        if action[i] < -1:
            action[i] = -1
            
        elif action[i] > 1:
            action[i] = 1
            
        action[i] = get_action(-0.01, 0, action[i])
        
    for i in range(3,6):
        if action[i] < -1:
            action[i] = -1
            
        elif action[i] > 1:
            action[i] = 1
            
        action[i] = get_action(0, 0.08, action[i])
        
    for i in range(6,8):
        if action[i] < -1:
            action[i] = -1
            
        elif action[i] > 1:
            action[i] = 1
            
        action[i] = get_action(0,0, action[i])
        
    if action[8] < -1:
        action[8] = -1
        
    elif action[8] > 1:
        action[8] = 1
        
    action[8] = get_action(0,0.08, action[8])
    
    action = action.reshape(3,3)
    
    return action

In [ ]:
def action_tile(action1, action2, action3, action4, action5, action6, action7, action8):
    
    action = np.zeros((num_jur, 3))
    
    action[cluster1_index,:] = action1
    action[cluster2_index,:] = action2
    action[cluster3_index,:] = action3
    action[cluster4_index,:] = action4
    action[cluster5_index,:] = action5
    action[cluster6_index,:] = action6
    action[cluster7_index,:] = action7
    action[cluster8_index,:] = action8
    
    return action

In [ ]:
def step(current_state, action1, action2, action3, action4, action5, action6, action7, action8):
    
    data_array = current_state[0]
    prep_values = current_state[9]
    current_time = current_state[10]

    action1 = change_action_range(action1)
    action2 = change_action_range(action2)
    action3 = change_action_range(action3)
    action4 = change_action_range(action4)
    action5 = change_action_range(action5)
    action6 = change_action_range(action6)
    action7 = change_action_range(action7)
    action8 = change_action_range(action8)
    
    a_unaware1 = action1[0]
    a_art1 = action1[1]
    a_prep1 = action1[2]
    
    a_unaware2 = action2[0]
    a_art2 = action2[1]
    a_prep2 = action2[2]
    
    a_unaware3 = action3[0]
    a_art3 = action3[1]
    a_prep3 = action3[2]
    
    a_unaware4 = action4[0]
    a_art4 = action4[1]
    a_prep4 = action4[2]
    
    a_unaware5 = action5[0]
    a_art5 = action5[1]
    a_prep5 = action5[2]
    
    a_unaware6 = action6[0]
    a_art6 = action6[1]
    a_prep6 = action6[2]
    
    a_unaware7 = action7[0]
    a_art7 = action7[1]
    a_prep7 = action7[2]
    
    a_unaware8 = action8[0]
    a_art8 = action8[1]
    a_prep8 = action8[2]
    
    
    a_unaware = action_tile(a_unaware1,a_unaware2,a_unaware3,a_unaware4,a_unaware5,a_unaware6,a_unaware7,a_unaware8)
    a_art = action_tile(a_art1, a_art2, a_art3, a_art4, a_art5, a_art6, a_art7, a_art8)
    a_prep = action_tile(a_prep1, a_prep2, a_prep3, a_prep4, a_prep5, a_prep6, a_prep7, a_prep8)
    
    #prep
    prep_rate = prep_values + a_prep
    
    pop_susceptible_12_years = data_array[:,:,0,0]
    
    total_reward = 0
    total_inf = 0
    total_cost = 0
    done = False
    
    total_pop, prevalence_prop, unaware_prop, aware_no_art_prop, aware_art_vls_prop,_ = \
        calculate_proportions(data_array, num_jur, number_of_risk_groups, unaware_index, aware_no_care_index, ART_VLS_index, VLS_index)

    new_inf_per_month = new_infections_per_month(num_jur, data_array, new_infections_data, M_x1_y1_i, prep_rate)
    death_per_month_risk_age_compartments = data_array*death_rate_risk_age_compartments*dt

    diagnosis_rate_risk = diagnosis_rate(data_array, num_jur, a_unaware, unaware_index, number_of_risk_groups, new_inf_per_month, unaware_prop, death_per_month_risk_age_compartments)

    dropout_rate_risk = dropout_rate(num_jur, a_art, ART_VLS_index, diagnosis_rate_risk, ltc_risk, gamma, number_of_risk_groups, data_array, new_inf_per_month, unaware_prop, aware_no_art_prop, aware_art_vls_prop, death_per_month_risk_age_compartments)

    Q_matrix = q_matrix(num_jur, new_infections_data, diagnosis_rate_risk, dropout_rate_risk, ltc_risk)

    Q_matrix_diagonal = q_mat_diag(Q_matrix, num_jur)
    
    
    for i in range(12):

        new_data = np.zeros((num_jur, number_of_risk_groups, age_groups, number_of_compartments))

        data_t_1 = data_array.copy()

        for risk in range(number_of_risk_groups):

            #calculate flow of infected to diff compartments and subtract from that compartment
            new_data[:,risk,:,:] = data_array[:,risk,:,:] + \
                                    np.matmul(data_array[:,risk,:,:], Q_matrix[:,risk,:,:]) - \
                                    np.matmul(data_array[:,risk,:,:], Q_matrix_diagonal[:,risk,:,:]) - \
                                    death_per_month_risk_age_compartments[:,risk,:,:]

            #subtract from susceptible and add to acute unaware
            new_data[:,risk,:,0] = new_data[:,risk,:,0] - new_inf_per_month[:,risk,:]
            
            new_data[:,risk,:,1] = new_data[:,risk,:,1] + new_inf_per_month[:,risk,:]

            #add the total deaths to last column
            new_data[:,risk,:,21] = np.sum(death_per_month_risk_age_compartments[:,risk,:,:], axis=2)


        cost_per_month = cost(data_t_1, new_data, unaware_prop, aware_art_vls_prop, diagnosis_rate_risk, dropout_rate_risk, prep_rate)

        benefit_per_month = benefit(new_data)

        reward_per_month = benefit_per_month - cost_per_month

        total_reward += reward_per_month
        
        total_cost += cost_per_month
        
        total_inf += new_inf_per_month

        data_array = new_data.copy()
    
    new_pop_dist = aging(data_array, pop_susceptible_12_years*(1+pop_growth_rate)) # adding new pop
    
    new_state1,new_state2,new_state3,new_state4,new_state5,new_state6,new_state7,new_state8 = extract_state(new_pop_dist, prep_rate)
                         
    next_state = (new_pop_dist,new_state1,new_state2,new_state3,new_state4,new_state5,new_state6,new_state7,new_state8, prep_rate, current_time+1)
    
    reward_cluster1 = total_reward[cluster1_index,:]
    reward_cluster2 = total_reward[cluster2_index,:]
    reward_cluster3 = total_reward[cluster3_index,:]
    reward_cluster4 = total_reward[cluster4_index,:]
    reward_cluster5 = total_reward[cluster5_index,:]
    reward_cluster6 = total_reward[cluster6_index,:]
    reward_cluster7 = total_reward[cluster7_index,:]
    reward_cluster8 = total_reward[cluster8_index,:]
    
#     reward1 = np.sum(reward_cluster1)
#     reward2 = np.sum(reward_cluster2)
#     reward3 = np.sum(reward_cluster3)
#     reward4 = np.sum(reward_cluster4)
#     reward5 = np.sum(reward_cluster5)
#     reward6 = np.sum(reward_cluster6)
#     reward7 = np.sum(reward_cluster7)
#     reward8 = np.sum(reward_cluster8)
    
    inf_cluster1 = total_inf[cluster1_index,:]
    inf_cluster2 = total_inf[cluster2_index,:]
    inf_cluster3 = total_inf[cluster3_index,:]
    inf_cluster4 = total_inf[cluster4_index,:]
    inf_cluster5 = total_inf[cluster5_index,:]
    inf_cluster6 = total_inf[cluster6_index,:]
    inf_cluster7 = total_inf[cluster7_index,:]
    inf_cluster8 = total_inf[cluster8_index,:]
    
    total_cost1 = total_cost[cluster1_index,:]
    total_cost2 = total_cost[cluster2_index,:]
    total_cost3 = total_cost[cluster3_index,:]
    total_cost4 = total_cost[cluster4_index,:]
    total_cost5 = total_cost[cluster5_index,:]
    total_cost6 = total_cost[cluster6_index,:]
    total_cost7 = total_cost[cluster7_index,:]
    total_cost8 = total_cost[cluster8_index,:]

    reward1 = -np.sum(inf_cluster1) 
    reward2 = -np.sum(inf_cluster2)  
    reward3 = -np.sum(inf_cluster3) 
    reward4 = -np.sum(inf_cluster4) 
    reward5 = -np.sum(inf_cluster5) 
    reward6 = -np.sum(inf_cluster6) 
    reward7 = -np.sum(inf_cluster7) 
    reward8 = -np.sum(inf_cluster8)
  

    if np.sum(total_cost1) > 2.00e8:
        reward1 -= (np.sum(total_cost1) - 2.00e8)
        
    if np.sum(total_cost2) > 7.89e8:
        reward2 -= (np.sum(total_cost2) - 7.89e8)
        
    if np.sum(total_cost3) > 2.00e8:
        reward3 -= (np.sum(total_cost3) - 2.00e8)
        
    if np.sum(total_cost4) > 1.28e8:
        reward4 -= (np.sum(total_cost4) - 1.28e8)
        
    if np.sum(total_cost5) > 2.00e8:
        reward5 -= (np.sum(total_cost5) - 2.00e8)
        
    if np.sum(total_cost6) > 1.28e8:
        reward6 -= (np.sum(total_cost6) - 1.28e8)
        
    if np.sum(total_cost7) > 2.56e8:
        reward7 -= (np.sum(total_cost7) - 2.56e8)
        
    if np.sum(total_cost8) > 3.53e9:
        reward8 -= (np.sum(total_cost8) - 3.53e9)
    


    if current_time+1 == 12:
        done = True
                         
        
    return next_state, reward1, reward2, reward3, reward4, reward5, reward6,reward7, reward8, done